# Resolva o problema proposto por:

**Métodos diretos:**
- Cramer;
- Eliminação de Gauss;
- Fatoração LU;
- Fatoração de Cholesky; (só no computador)

**Métodos iterativos:**
- Gauss-Siedel.

# Problema
Determinar a corrente em cada resistor do circuito abaixo:

![](./assets/circuito.png)

# Solução

Para solucionar o circuito a seguir, sera utilizada a lei de Kirchhof, e sera representado na forma matricial, e com os metodos estudados sera resolvido o sistema

$$
I_{1} * 10 + I_{1} * 5 + I_{1} * 15 + (I_{1} - I_{2}) * 10 = 0 \\
I_{2} * 5 + (I_{2} - I_{1}) * 10 + I_{2} * 20 = 200 \\
$$

$$
I_{23} = I_{34} = I_{45} = I_{1} \\
I_{12} = I_{56} = I_{2} \\
I_{25} = I_{2} - I_{1} \\
$$

Simplificando:
$$
40I_{1} - 10I_{2} = 0 \\
-10I_{1} + 35I_{2} = 200 \\
$$
Colocando na forma matricial:
$$
\begin{bmatrix}
40 & -10 \\
-10 & 35 \\
\end{bmatrix}
\begin{bmatrix}
I_{1} \\
I_{2} \\
\end{bmatrix}
=
\begin{bmatrix}
0 \\
200 \\
\end{bmatrix}
$$






### Definindo a matriz em codigo

In [1]:
import numpy as np

# Coefficient matrix
a = np.array(
	[
		[40, -10],
		[-10, 35]
	]
)

# Independent terms
b = np.array(
	[
		[0],
		[200]
	]
)


### Cramer

In [3]:
import numpy as np

def crammer(a: np.ndarray , b: np.ndarray) -> np.array:
	a = a.astype(float)  # Ensure a is a float matrix
	b = b.astype(float)  # Ensure b is a float vector

	# Determinant of the coefficient matrix
	det = np.linalg.det(a)

	# Create the variable vector
	x = np.zeros(len(a))

	# For column in the coefficient matrix
	for i in range(len(a)):
		# Copy the coefficient matrix
		a_c = np.copy(a)

		# Replace the column with the independent terms
		a_c[:, i] = b[:, 0]

		# Calculate the determinant of the new matrix
		det_c = np.linalg.det(a_c)

		# Calculate the value of the variable
		x[i] = det_c / det
		
	# Return the variable vector
	return x

print("Original Matrix a:")
print(a)

print("Original Vector b:")
print(b)

x = crammer(a, b)
print("Solution x:")
print(x)


Original Matrix a:
[[ 40 -10]
 [-10  35]]
Original Vector b:
[[  0]
 [200]]
Solution x:
[1.53846154 6.15384615]


### Eliminação de Gauss

In [4]:
import numpy as np

# adapted from https://en.wikipedia.org/wiki/Gaussian_elimination#Pseudocode
def gauss_elimination(a: np.ndarray, b: np.ndarray) -> np.array:
	a = a.astype(float)  # Ensure a is a float matrix
	b = b.astype(float)  # Ensure b is a float vector
	
	# Create a copy of a to avoid modifying the original matrix
	a_copy = np.copy(a)
	
	# Create an augmented matrix ab by stacking a and b horizontally
	ab = np.hstack([a_copy, b.reshape(-1, 1)])

	m, n = a.shape  # Gest the number of rows (m) and columns (n)
	h = 0  # Initialization of the pivot row
	k = 0  # Initialization of the pivot column

	while h < m and k < n:
		# Find the k-th pivot
		i_max = np.argmax(np.abs(ab[h:m, k])) + h
		if ab[i_max, k] == 0:
			# No pivot in this column, pass to next column
			k += 1
		else:
			# Swap rows h and i_max
			ab[[h, i_max]] = ab[[i_max, h]]

			# Do for all rows below pivot
			for i in range(h + 1, m):
				f = ab[i, k] / ab[h, k]
				
				# Fill with zeros the lower part of pivot column
				ab[i, k] = 0
				
				# Do for all remaining elements in current row
				for j in range(k + 1, n + 1):  # n+1 to include the last column of ab
					ab[i, j] -= ab[h, j] * f

			# Increase pivot row and column
			h += 1
			k += 1

	# back-substitution to find the solution x
	x = np.zeros(m)
	for i in range(m - 1, -1, -1):
		x[i] = (ab[i, -1] - np.dot(ab[i, i+1:n], x[i+1:])) / ab[i, i]

	return x


print("Original Matrix a:")
print(a)

print("Original Vector b:")
print(b)

x = gauss_elimination(a, b)
print("Solution x:")
print(x)

Original Matrix a:
[[ 40 -10]
 [-10  35]]
Original Vector b:
[[  0]
 [200]]
Solution x:
[1.53846154 6.15384615]


### Fatoração LU
Para a fatoração LU sera utilizado o metodo Doolittle's
Algumas referencias:
- [https://www.youtube.com/watch?v=C1aBCvn7CUU](https://www.youtube.com/watch?v=C1aBCvn7CUU)
- [https://en.wikipedia.org/wiki/LU_decomposition#Doolittle_algorithm](https://en.wikipedia.org/wiki/LU_decomposition#Doolittle_algorithm)

In [5]:
import numpy as np

# adapted from https://en.wikipedia.org/wiki/LU_decomposition#Code_examples
import numpy as np

def lu_doolittle(a):
	a = a.astype(float)  # Ensure a is a float matrix
	n = len(a)
	LU = np.copy(a).astype(float)
	# Using Doolittle's Method (for more information see https://en.wikipedia.org/wiki/LU_decomposition#Doolittle_algorithm)
	for i in range(n):
		for j in range(i):
			LU[i, j] = (LU[i, j] - np.dot(LU[i, :j], LU[:j, j])) / LU[j, j]
		j = np.arange(i, n)
		LU[i, j] = LU[i, j] - np.dot(LU[i, :i], LU[:i, j])
	return LU

def solve_linear_system(LU, b):
	b = b.astype(float)  # Ensure b is a float vector
	n = len(LU)
	y = np.zeros_like(b, dtype=float)
	# Find solution of Ly = b
	for i in range(n):
		y[i, :] = b[i, :] - np.dot(LU[i, :i+1], y[:i+1, :])
	# Find solution of Ux = y
	x = np.zeros_like(b, dtype=float)
	for i in range(n-1, -1, -1):
		x[i, :] = (y[i, :] - np.dot(LU[i, i+1:], x[i+1:, :])) / LU[i, i]
	return x


print("Original Matrix a:")
print(a)

print("Original Vector b:")
print(b)

lu = lu_doolittle(a)

print("LU decomposition of a:")
print(lu)

x = solve_linear_system(lu, b)

print("Solution x:")
print(x)

Original Matrix a:
[[ 40 -10]
 [-10  35]]
Original Vector b:
[[  0]
 [200]]
LU decomposition of a:
[[ 40.   -10.  ]
 [ -0.25  32.5 ]]
Solution x:
[[1.53846154]
 [6.15384615]]


### Fatoração de Cholesky

In [6]:
# Cholesky decomposition

import numpy as np

def cholesky_decomposition(a):
	a = a.astype(float)  # Ensure a is a float matrix
	# Check if the matrix is square
	if a.shape[0] != a.shape[1]:
		raise ValueError("The input matrix must be square.")

	
	n = a.shape[0]
	L = np.zeros((n, n))

	for i in range(n):
		for j in range(i+1):
			s = sum(L[i, k] * L[j, k] for k in range(j))

			if i == j:
				# Diagonal elements
				L[i, i] = np.sqrt(a[i, i] - s)
			else:
				# Off-diagonal elements
				L[i, j] = (1.0 / L[j, j]) * (a[i, j] - s)

	return L

# Solve a linear system using the Cholesky decomposition

def solve_linear_system(L, b):
	b = b.astype(float)  # Ensure b is a float vector
	n = L.shape[0]
	y = np.zeros_like(b, dtype=float)
	# Find solution of Ly = b
	for i in range(n):
		y[i, :] = (b[i, :] - np.dot(L[i, :i+1], y[:i+1, :])) / L[i, i]
	# Find solution of LTx = y
	x = np.zeros_like(b, dtype=float)
	for i in range(n-1, -1, -1):
		x[i, :] = (y[i, :] - np.dot(L[i+1:, i], x[i+1:, :])) / L[i, i]
	return x

print("Original Matrix a:")
print(a)

print("Original Vector b:")
print(b)

l = cholesky_decomposition(a)

print("Cholesky decomposition of a:")
print(l)

x = solve_linear_system(l, b)

print("Solution x:")
print(x)

Original Matrix a:
[[ 40 -10]
 [-10  35]]
Original Vector b:
[[  0]
 [200]]
Cholesky decomposition of a:
[[ 6.32455532  0.        ]
 [-1.58113883  5.70087713]]
Solution x:
[[1.53846154]
 [6.15384615]]


### Gauss-Siedel

In [7]:
# Gauss-Seidel method

import numpy as np

def gauss_seidel(a, b, x0, tol=1e-6, max_iter=100):
	a = a.astype(float)  # Ensure a is a float vector
	b = b.astype(float)  # Ensure b is a float vector
	x0 = x0.astype(float)  # Ensure x0 is a float vector
	# Check if the matrix is square
	if a.shape[0] != a.shape[1]:
		raise ValueError("The input matrix must be square.")

	# Check if the matrix is symmetric
	if not np.all(a == a.T):
		raise ValueError("The input matrix must be symmetric.")

	# Check if the matrix is positive definite
	if not np.all(np.linalg.eigvals(a) > 0):
		raise ValueError("The input matrix must be positive definite.")

	# Number of rows/columns
	n = a.shape[0]

	# Initial guess
	x = x0.copy()

	# Iterate until convergence
	for _ in range(max_iter):
		for i in range(n):
			# Split the matrix into the two parts a = L + U
			L = np.tril(a, k=-1)
			U = np.triu(a, k=1)

			# Calculate the next iteration
			x[i] = (b[i] - np.dot(L[i, :], x) - np.dot(U[i, :], x)) / a[i, i]

		# Check if the solution has converged
		if np.linalg.norm(np.dot(a, x) - b) < tol:
			break

	return x

print("Original Matrix a:")
print(a)

print("Original Vector b:")
print(b)

x = gauss_seidel(a, b, np.zeros_like(b))

print("Solution x:")
print(x)


Original Matrix a:
[[ 40 -10]
 [-10  35]]
Original Vector b:
[[  0]
 [200]]
Solution x:
[[1.53846152]
 [6.15384615]]
